In [3]:
!pip install git+https://github.com/StatMixedML/XGBoostLSS.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/StatMixedML/XGBoostLSS.git to /private/var/folders/m5/qjbw9mwx66n97zmrlsfzns380000gn/T/pip-req-build-sn498swt
  Running command git clone --filter=blob:none --quiet https://github.com/StatMixedML/XGBoostLSS.git /private/var/folders/m5/qjbw9mwx66n97zmrlsfzns380000gn/T/pip-req-build-sn498swt
  Resolved https://github.com/StatMixedML/XGBoostLSS.git to commit 03a860da4de1a6a4d6c2db55be879bf6ea681f36
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xgboostlss to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following yanked versions: 7.1.0, 7.30.0, 8.13.

In [4]:
!pip install fbm

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbm import FBM
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

import xgboost as xgb
from xgboostlss.model import XGBoostLSS
from xgboostlss.distributions.Gaussian import Gaussian

import multiprocessing
import plotnine
from plotnine import *
plotnine.options.figure_size = (12, 8)

from xgboostlss.distributions.Gaussian import Gaussian

# fBmデータをプロットする関数
def plot_fbm(data, H):
    plt.figure(figsize=(10, 6))
    plt.plot(data, label=f'fBm with H={H}')
    plt.title(f'Fractional Brownian Motion with H={H}')
    plt.xlabel('Time Step')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()

# 分数ブラウン運動のデータを生成する関数
def generate_fbm(H, N):
    fbm = FBM(n=N-1, hurst=H, length=1, method='daviesharte')
    fbm_samples = fbm.fbm()
    return fbm_samples

# データを準備する関数
def prepare_data(data, past_steps, future_steps):
    X = []
    y_mean = []
    y_std = []
    total_steps = past_steps + future_steps
    for i in range(len(data) - total_steps):
        past_data = data[i:i+past_steps]
        future_data = data[i+past_steps:i+total_steps]
        X.append(past_data)
        y_mean.append(np.mean(future_data))
        y_std.append(np.std(future_data))
    return np.array(X), np.array(y_mean), np.array(y_std)

ModuleNotFoundError: No module named 'xgboostlss'

In [ ]:
# パラメータ設定
H = 0.8  # Hurst指数
N = 10000  # サンプル数
past_steps = 128
future_steps = 128

# fBmデータ生成と前処理
data = generate_fbm(H, N)
X, y_mean, y_std = prepare_data(data, past_steps, future_steps)
plot_fbm(data, H)

# データセットの分割
X_train, X_test, y_train_mean, y_test_mean, y_train_std, y_test_std = train_test_split(
    X, y_mean, y_std, test_size=0.2, random_state=42
)

# モデルパラメータの設定
params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'eta': 0.01,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'nthread': -1
}

# Gaussian 分布を指定してモデルを作成
model = XGBoostLSS(dist=Gaussian())

# モデルのトレーニング
dtrain = xgb.DMatrix(X_train, label=y_train_mean)
dtest = xgb.DMatrix(X_test, label=y_test_mean)
model.train(params=params, dtrain=dtrain, num_boost_round=100)

# 推定
predictions = model.predict(dtest)
y_pred_mean = predictions['loc']
y_pred_std = predictions['scale']

# 結果の評価と可視化
plt.figure(figsize=(10, 5))
plt.plot(y_test_mean, label="True Mean")
plt.plot(y_pred_mean, label="Predicted Mean", linestyle="--")
plt.title("Predicted vs. True Mean of Future Data")
plt.xlabel("Sample")
plt.ylabel("Mean Value")
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(y_test_std, label="True Std Dev")
plt.plot(y_pred_std, label="Predicted Std Dev", linestyle="--")
plt.title("Predicted vs. True Standard Deviation of Future Data")
plt.xlabel("Sample")
plt.ylabel("Standard Deviation")
plt.legend()
plt.show()


from sklearn.metrics import mean_squared_error
# 平均のMSEを計算
mse_mean = mean_squared_error(y_test_mean, y_pred_mean)
print("Mean Squared Error for Mean Prediction:", mse_mean)
# 標準偏差のMSEを計算
mse_std = mean_squared_error(y_test_std, y_pred_std)
print("Mean Squared Error for Standard Deviation Prediction:", mse_std)